In [ ]:
import os, pandas as pd, networkx as nx, numpy as np, statsmodels.api as sm
from sqlalchemy import create_engine
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt, seaborn as sns

## parameters

In [ ]:
yeari, yearf = '2024', '2024'
weeki, weekf = '18', '31'

In [ ]:
di = datetime.strptime(f'{yeari}-{weeki}-1', "%Y-%W-%w").date()
df = datetime.strptime(f'{yearf}-{weekf}-1', "%Y-%W-%w").date() + timedelta(6)
ds = [di+timedelta(dt) for dt in range((df-di).days+1)]
daylist = ds
print(di, 'until', df)

In [ ]:
cdef = 'tl7_10m'# 'tl5_10m' 'tl6_10m' 'tl7_10m' 'tl8_10m' 'tl8_60m'
cdef_alt = '16m_10min'# tl5: 62 ... tl7: 16   tl8: 8

## load data

In [ ]:
# load stadium meta data
stadname_data = pd.read_csv('output/00_stadname_data.csv')
aid2name = {aid: name for aid, name in zip(stadname_data.area_id, stadname_data.area_name)}

aid2city = {aid: city for aid, city in sorted({
    1: 'München',
    2: 'Berlin',
    3: 'Leipzig',
    4: 'Frankfurt am Main',
    10: 'Dortmund',
    11: 'Stuttgart',
    15: 'Köln',
    24: 'Düsseldorf',
    25: 'Hamburg',
    28: 'Gelsenkirchen',

    16: 'Berlin',
    29: 'Hamburg',
    41: 'München',
    64: 'Frankfurt am Main',
}.items(), key=lambda item: item[1])}
city2aid = {city: aid for aid, city in [(aid, city) for aid, city in aid2city.items()][::-1]}

In [ ]:
# n_c Germany & stadiums: load total contact numbers for Germany and stadiums
data_germany = pd.read_csv(f'output/00_ncontacts_germany_{cdef}.csv')
data_germany['day'] = [d.date() for d in pd.to_datetime(data_germany.day)]

# n_c cities: load total contact numbers numbers for cities
data = pd.read_csv(f'output/00_ncontacts_cities_{cdef}.csv')
data['day'] = [d.date() for d in pd.to_datetime(data.day)]

# n_pop cities & stadiums: load stadium capacity data for EURO 2024
stadium_data = pd.read_csv('output/00_stadium_data.csv')

# n_d Germany: load did numbers for Germany
panel_data_germany = pd.read_csv('output/00_panel_data_germany.csv')
panel_data_germany['day'] = [d.date() for d in pd.to_datetime(panel_data_germany.day)]

data_germany = data_germany.merge(panel_data_germany, on='day')

# n_d cities: load did numbers for cities
panel_data = pd.read_csv('output/00_panel_data.csv')
panel_data = panel_data.merge(stadium_data[['city','population']])
panel_data['pdid'] = panel_data.ndids / panel_data.population

# n_d stadiums: load did numbers for stadiums
panelstad_data2 = pd.read_csv('output/00_panelstad_data2.csv')
panelstad_data2['day'] = [d.date() for d in pd.to_datetime(panelstad_data2.day)]

# n_p cities: load ping numbers per did for cities
data_pingfreq = pd.read_csv('output/00_data_pingfreq.csv')
data_pingfreq['fpingsperdid'] = data_pingfreq.npingsperdid / 144.

# load mass event data
match_data = pd.read_csv('output/00_event_data.csv')
match_data['day'] = [d.date() for d in pd.to_datetime(match_data.day)]
match_data['capacity'] = [float(cap) if cap!='auto' else cap for cap in match_data.capacity]

In [ ]:
vac_data = [
    ['Berlin', date(2024,7,18), date(2024,8,30)],
    ['Dortmund', date(2024,7,8), date(2024,8,20)],
    ['Düsseldorf', date(2024,7,8), date(2024,8,20)],
    ['Frankfurt am Main', date(2024,7,15), date(2024,8,23)],
    ['Gelsenkirchen', date(2024,7,8), date(2024,8,20)],
    ['Hamburg', date(2024,7,18), date(2024,8,28)],
    ['Köln', date(2024,7,8), date(2024,8,20)],
    ['Leipzig', date(2024,6,20), date(2024,8,2)],
    ['München', date(2024,7,29), date(2024,9,9)],
    ['Stuttgart', date(2024,7,25), date(2024,9,7)],
]
vac_data = pd.DataFrame(vac_data, columns=['city','day_start','day_end'])
#vac_data

## event comparison

In [ ]:
def match2event(match):
    if '-' in match and len(match)==7:
        event = match
    else:
        mapping = {
            'R Kaiser': 'Roland Kaiser',
            'P Maffay': 'Peter Maffay',
            'A Gabalier': 'Andreas Gabalier',
            'T Swift': 'Taylor Swift',
            'T Scott': 'Travis Scott',
            'H Carpendale': 'Howard Carpendale',
            'M M-Westernhagen': 'Marius Müller Westernhagen',
            'Death Punch': 'Five Finger Death Punch',
            'B Ceylan': 'Bülent Ceylan',
            'A Rieu': 'Andre Rieu',
            'Stadtfest': 'Leipzig Stadtfest',
            'Weinfest': 'Leipzig Weinfest',
            'Hafengeburtstag': 'Hamburg Hafengeburtstag',
            'Rundfunk Open Air': 'Berliner Rundfunk Open Air',
            'CSD': 'Christopher Street Day',
            'BLiga': 'Bundesliga',
            'BLiga rel': 'Bundesliga relegation',
            'UEFA CL': 'UEFA Champions League',
            'Am. Football': 'American football'
        }
        if match in mapping:
            event = mapping[match]
        else:
            event = match
    return event

In [ ]:
match_rank = match_data\
                .merge(panel_data[['city','pdid','ndids']], on='city', how='left')\
                .merge(stadium_data[['area_id','population']], on='area_id', how='left')\
                .merge(data[['day','city','area_id','ncontacts_2']], on=['day','city','area_id'], how='left')\
                .merge(data[['day','city','ncontacts_1']].groupby(['day','city']).sum().reset_index(), on=['day','city'], how='left')\
                .merge(panelstad_data2, on=['day','area_id'], how='left', suffixes=('','_stad'))\
                .merge(data_pingfreq[['city','fpingsperdid']], on='city')\
                .drop_duplicates()
match_rank['pdid_stad'] = match_rank.ndids_stad / match_rank.capacity
match_rank

In [ ]:
expo_city_p, expo_city_q = 0.9422, 3.5642
expo_stad_p, expo_stad_q = 1.5769, 0.4554# for tl7 10min   1.1038, 1.5373# for tl8 60min

In [ ]:
print(f'expo_city_p: {expo_city_p}, expo_city_q: {expo_city_q}')
print(f'expo_stad_p: {expo_stad_p}, expo_stad_q: {expo_stad_q}')

match_rank['ncontacts_1'] = match_rank.ncontacts_1 / match_rank.ndids / match_rank.fpingsperdid**(expo_city_q) # / match_rank.pdid**(expo_city_p-1.)

ncthr = 20
match_rank['ncontacts_2'] = [n if n>=ncthr else np.nan for n in match_rank.ncontacts_2]
match_rank['ncontacts_2'] = match_rank.ncontacts_2 / match_rank.ndids_stad / (1e-16+match_rank.pdid_stad)**(expo_stad_p-1.) / match_rank.fpingsperdid**(expo_city_q)

match_rank = match_rank.merge(data_germany, on='day', how='left', suffixes=('', '_germany'))
match_rank['ncontacts_3'] = match_rank.ncontacts_1_germany / match_rank.ndid_1 / data_pingfreq.fpingsperdid.mean()**(expo_city_q) # / data_germany.pdid_1.mean()

match_rank['event'] = [match2event(match) for match in match_rank.match]
match_rank_aux = match_rank.copy(deep=True)
match_rank = match_rank[['city','event','ncontacts_2','ncontacts_1','ncontacts_3']]
match_rank.loc[:,'ncontacts_1'] = [np.log10(n) if n>0 else np.nan for n in match_rank.ncontacts_1]
match_rank.loc[:,'ncontacts_2'] = [np.log10(n) if n>0 else np.nan for n in match_rank.ncontacts_2]
match_rank.loc[:,'ncontacts_3'] = [np.log10(n) if n>0 else np.nan for n in match_rank.ncontacts_3]

In [ ]:
to_exclude = ['A Bayern','Bundesliga relegation','American football','test match','RLiga (f)','HLiga','BLiga3','HipHop','S25 Berlin',
              'UEFA Champions League','B2Run','DFB Pokal','friendly match']
to_plot = match_rank[~match_rank.event.isin(to_exclude)].sort_values(['ncontacts_1','ncontacts_2','ncontacts_3'], ascending=False)

In [ ]:
class2event = {
    'convention': ['DoKomi','Dogs & Fun'],
    'concert Hard-Rock/Metal': ['AC/DC','Rammstein','Five Finger Death Punch','Metallica'],
    "concert Rock'n'Roll/Pop": ['Peter Maffay','SDP','P!NK','Andre Rieu','Andreas Gabalier','Taylor Swift','Green Day','Coldplay',
                                'Berliner Rundfunk Open Air'],
    'concert Schlager': ['Howard Carpendale','Schlagerfest XXL','Marius Müller Westernhagen','Roland Kaiser','Schlagernacht',
                         'Dortmund OLE'],
    'concert Rap': ['Bushido','Travis Scott','Apache 207'],
    'festival': ['Leipzig Stadtfest','Christopher Street Day','Japan Day','Hamburg Hafengeburtstag','Rheinkirmes','Tollwood',
                 'Leipzig Weinfest','Schalke Tach'],
    'Bundesliga': ['Bundesliga'],
    'comedy': ['Bülent Ceylan','Mario Barth'],
}
event2class = {}
for cl, events in class2event.items():
    for event in events:
        event2class[event] = cl
for event in set(to_plot.event):
    if event not in event2class:
        if 'GER' in event:
            event2class[event] = 'EURO 2024 incl. GER'
        else:
            event2class[event] = 'EURO 2024 excl. GER'
to_plot['event_class'] = to_plot.event.map(event2class)

In [ ]:
# Make the PairGrid

sns.set_theme(style="ticks")
g = sns.PairGrid(to_plot,
                 x_vars=['ncontacts_2','ncontacts_1','ncontacts_3'], y_vars=["event"],
                 height=12.5, aspect=.24)

# Draw a dot plot using the stripplot function
g.map(sns.stripplot, size=7.5, orient="h", jitter=False,# alpha=.5,
      linewidth=0, edgecolor="w", hue=to_plot['city'], hue_order=sorted(set(list(aid2city.values()))),
      palette=sns.hls_palette(len(set(match_data.city))))# palette="flare_r", husl/hls_palette

# Use the same x axis limits on all columns and add better labels
#g.set(xlim=(0, 25), xlabel="contacts", ylabel="")

# Use semantically meaningful titles for the columns
titles = ["log10(contacts/person host stadium)", "log10(contacts/person host city)", "log10(contacts/person Germany)"]

for ax, title in zip(g.axes.flat, titles):

    # Set a different title for each axes
    #ax.set(title=title)

    # Make the grid horizontal instead of vertical
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)

# Access the axes and set properties per column
xmin = min(match_rank[match_rank.ncontacts_1>0].ncontacts_1.min(), match_rank[match_rank.ncontacts_2>0].ncontacts_2.min(),
           match_rank[match_rank.ncontacts_2>0].ncontacts_3.min())
xmax = max(match_rank.ncontacts_1.max(), match_rank.ncontacts_2.max(), match_rank.ncontacts_3.max())
dx = .1
for i, ax_col in enumerate(g.axes):
    for j, ax in enumerate(ax_col):
        # Example: Set specific properties per column
        if j == 0:  # First column (x-axis)
            ax.set_xlim([max(0,xmin)-dx, xmax+dx])
            ax.set_ylabel('event')
            ax.set_xlabel(titles[0])
        elif j == 1:
            ax.set_xlim([max(0,xmin)-dx, xmax+dx])
            ax.set_xlabel(titles[1])
        elif j == 2:
            ax.set_xlim([max(0,xmin)-dx, xmax+dx])
            ax.set_xlabel(titles[2])

g.add_legend(title='host city')
sns.despine(left=True, bottom=True)

plt.savefig(f'plots/fig2_{cdef_alt}/05_eventcmp_horvitz.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/05_eventcmp_horvitz.pdf', bbox_inches='tight')
plt.show()

In [ ]:
sns.set_theme(style="ticks")
g = sns.PairGrid(to_plot,
                 x_vars=['ncontacts_2','ncontacts_1','ncontacts_3'], y_vars=["event_class"],
                 height=2.5, aspect=1.2)

# Draw a dot plot using the stripplot function
g.map(sns.stripplot, size=7.5, orient="h", jitter=False,# marker='|',# alpha=.5,
      linewidth=0, edgecolor="w", hue=to_plot['city'], hue_order=sorted(set(list(aid2city.values()))),
      palette=sns.hls_palette(len(set(match_data.city))))# palette="flare_r", husl/hls_palette

# Use the same x axis limits on all columns and add better labels
#g.set(xlim=(0, 25), xlabel="contacts", ylabel="")

# Use semantically meaningful titles for the columns
titles = ["log10(contacts/person host stadium)", "log10(contacts/person host city)", "log10(contacts/person Germany)"]

for ax, title in zip(g.axes.flat, titles):

    # Set a different title for each axes
    #ax.set(title=title)

    # Make the grid horizontal instead of vertical
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)

# Access the axes and set properties per column
xmin = min(match_rank[match_rank.ncontacts_1>0].ncontacts_1.min(), match_rank[match_rank.ncontacts_2>0].ncontacts_2.min(),
           match_rank[match_rank.ncontacts_2>0].ncontacts_3.min())
xmax = max(match_rank.ncontacts_1.max(), match_rank.ncontacts_2.max(), match_rank.ncontacts_3.max())
dx = .1
for i, ax_col in enumerate(g.axes):
    for j, ax in enumerate(ax_col):
        # Example: Set specific properties per column
        if j == 0:  # First column (x-axis)
            ax.set_xlim([max(0,xmin)-dx, xmax+dx])
            ax.set_ylabel('event category')
            ax.set_xlabel(titles[0])
        elif j == 1:
            ax.set_xlim([max(0,xmin)-dx, xmax+dx])
            ax.set_xlabel(titles[1])
        elif j == 2:
            ax.set_xlim([max(0,xmin)-dx, xmax+dx])
            ax.set_xlabel(titles[2])

g.add_legend(title='host city')
sns.despine(left=True, bottom=True)

plt.savefig(f'plots/fig2_{cdef_alt}/05_eventcmp_horvitz_categorized.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig2_{cdef_alt}/05_eventcmp_horvitz_categorized.pdf', bbox_inches='tight')
plt.show()

- GER-DEN: rainfall https://www.welt.de/sport/fussball/em/article252271302/Deutschland-Daenemark-Diese-Bilder-gingen-um-die-Welt.html
- ENG-NED: rainfall https://www.sportschau.de/fussball/uefa-euro-2024/regenfaelle-in-dortmund-vor-em-halbfinale,regen-dortmund-100.html
- ROM-UKR: demo https://www.br.de/nachrichten/bayern/emotionaler-em-auftakt-fuer-die-ukraine-in-muenchen,UFy5LuC
- TUR-GEO: rainfall https://www.welt.de/sport/fussball/em/article252096768/Tuerkei-Georgien-Das-bislang-groesste-Spektakel-der-EM-2024.html
- ENG-SRB: riot https://www.welt.de/sport/fussball/em/article252059140/EM-2024-Vor-dem-Spiel-geraten-Fans-von-England-und-Serbien-aneinander.html
- all Gelsenkirchen matches after ENG-SRB except ENG-SLO: fear of riot https://uefaeuro2024.gelsenkirchen.de/de/index.aspx
- DEN-ENG: city center crowded https://www.hessenschau.de/panorama/daenemark---england-roemerberg-in-frankfurt-in-britischer-hand-prinz-william-im-stadion-v9,em-frankfurt-frederik-100.html
- SVK-ROM: storm, fan zone visitors sent home https://www.hessenschau.de/panorama/em-fanmeile-in-frankfurt-wegen-unwetter-geschlossen-v2,kurz-frankfurt-fanmeile-geschlossen-unwetter-100.html
- FRA-BEL: fan march https://www1.wdr.de/nachrichten/rheinland/duesseldorf-em-fanmaersche-belgien-frankreich-100.html
- ALB-ESP: mobile data record https://rp-online.de/sport/fussball/em/fussball-em-2024-datenrekord-bei-spanien-gegen-albanien-in-duesseldorfer-arena_aid-115042555

In [ ]:
cols = ['day','city','event','ncontacts_2','ncontacts_1','ncontacts_3']
reg2reg = {f'ncontacts_{reg}': f'ncontacts_{name}' for reg, name in zip([1,2,3], ['hostcity', 'hoststad', 'germany'])}
for_andrzej = match_rank_aux[(match_rank_aux.day>=di) & (match_rank_aux.day<=df)][cols]\
    .sort_values('day')\
    .rename(columns=reg2reg)\
    .reset_index(drop=True)

In [ ]:
for_andrzej.to_csv(f'output/05_event_comparison_{cdef_alt}.csv', index=False, encoding='utf-8-sig')#f'data/fig2_{cdef_alt}/event_comparison.csv'

## baseline comparison (as in Fig. 1, but Horvitz-corrected)

In [ ]:
match_rank = data[['day','city','area_id']]\
                .merge(panel_data[['city','pdid','ndids']], on='city', how='left')\
                .merge(stadium_data[['city','population']], on='city', how='left')\
                .merge(data[['day','city','area_id','ncontacts_2']], on=['day','city','area_id'], how='left')\
                .merge(data[['day','city','ncontacts_1']].groupby(['day','city']).sum().reset_index(), on=['day','city'], how='left')\
                .merge(panelstad_data2, on=['day','area_id'], how='left', suffixes=('','_stad'))\
                .merge(match_data, on=['day','city','area_id'], how='left')\
                .merge(data_pingfreq[['city','fpingsperdid']], on='city')\
                .drop_duplicates()
match_rank['pdid_stad'] = match_rank.ndids_stad / match_rank.capacity
match_rank

In [ ]:
print(f'expo_city_p: {expo_city_p}, expo_city_q: {expo_city_q}')
print(f'expo_stad_p: {expo_stad_p}, expo_stad_q: {expo_stad_q}')

match_rank['ncontacts_1'] = match_rank.ncontacts_1 / match_rank.ndids / match_rank.fpingsperdid**(expo_city_q) # / match_rank.pdid**(expo_city_p-1.)
match_rank['ncontacts_1'] = match_rank.ncontacts_1 * match_rank.population/1e4

ncthr = 20
match_rank['ncontacts_2'] = [n if n>=ncthr else np.nan for n in match_rank.ncontacts_2]
match_rank['ncontacts_2'] = match_rank.ncontacts_2 / match_rank.ndids_stad / (1e-16+match_rank.pdid_stad)**(expo_stad_p-1.) / match_rank.fpingsperdid**(expo_city_q)
match_rank['ncontacts_2'] = match_rank.ncontacts_2 * match_rank.capacity/1e4

#match_rank = match_rank.merge(data_germany, on='day', how='left', suffixes=('', '_germany'))
#match_rank['ncontacts_3'] = match_rank.ncontacts_1_germany / match_rank.ndid_1 / data_pingfreq.fpingsperdid.mean()**(expo_city_q) # / data_germany.pdid_1.mean()

#match_rank['event'] = [match2event(match) for match in match_rank.match]
#match_rank_aux = match_rank.copy(deep=True)
#match_rank = match_rank[['city','event','ncontacts_2','ncontacts_1','ncontacts_3']]
#match_rank.loc[:,'ncontacts_1'] = [np.log10(n) if n>0 else np.nan for n in match_rank.ncontacts_1]
#match_rank.loc[:,'ncontacts_2'] = [np.log10(n) if n>0 else np.nan for n in match_rank.ncontacts_2]
#match_rank.loc[:,'ncontacts_3'] = [np.log10(n) if n>0 else np.nan for n in match_rank.ncontacts_3]

In [ ]:
data_n = match_rank
data_n

In [ ]:
#data_here = data.copy(deep=True)
data_1 = data_n.drop(columns=['area_id']).groupby(['day','city']).ncontacts_1.sum().reset_index()
data_2 = data_n[data_n.area_id>-.5].drop(columns=['area_id']).groupby(['day','city']).ncontacts_2.sum().reset_index()
data_here = data_1.merge(data_2, on=['day','city'], how='left')
data_here['ncontacts'] = data_here.ncontacts_1 + data_here.ncontacts_2.fillna(0.)
data_here['wd'] = [d.weekday() for d in data_here.day]

# baseline = number contacts in city without stadiums by weekday, averaged over many weeks
baseline = pd.DataFrame(data_here.groupby(['city','wd']).ncontacts_1.mean()).rename(columns={'ncontacts_1':'baseline'}).reset_index()
data_here = data_here.merge(baseline, on=['city','wd'])

data_here2, data_here3 = data_here.copy(deep=True), data_here.copy(deep=True)
data_here2['to_baseline'] = data_here.ncontacts_1/data_here.baseline
data_here2['with_stadiums'] = False
data_here3['to_baseline'] = (data_here.ncontacts_1+data_here.ncontacts_2.fillna(0.))/data_here.baseline
data_here3['with_stadiums'] = True
data_here4 = pd.concat([data_here2, data_here3], ignore_index=True)

#baseline_ci = pd.DataFrame(data_here.groupby(['city','wd']).ncontacts_1.std()).rename(columns={'ncontacts_1':'baseline_std'}).reset_index()
#baseline_ci['lower'] = 1. - baseline_ci.baseline_std / baseline.baseline
#baseline_ci['upper'] = 1. + baseline_ci.baseline_std / baseline.baseline

data_here4

In [ ]:
sns.set_theme(style="ticks")

# Define the palette as a list to specify exact values
palette = sns.husl_palette(2)#sns.color_palette("rocket_r")

city_list = sorted(set(list(aid2city.values()))) #set(data_here.city)

# Plot the lines on two facets
g = sns.relplot(
    data=data_here4,
    x="day", y="to_baseline",
    hue="with_stadiums", hue_order=[True,False],
    row="city", row_order=city_list,# size="choice",# col="align",
    kind="line", palette=palette,# size_order=[False, True], 
    height=2, aspect=5.5, facet_kws=dict(sharex=False),
)

ylimup = 70
axes = g.axes
holis = [date(2024,5,1),date(2024,5,9), date(2024,5,20)]
for ax_row, city in zip(axes, city_list):
    for ax in ax_row:
        ax.set_yscale('log')
        ax.set_ylabel('contacts')
        ax.plot([data_here.day.min(), data_here.day.max()], [1,1], c='gray')
        ax.set_title(city)
        ax.set_xticks(list(set(data_here.day)))
        ax.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in list(set(data_here.day))])#, rotation=90)
        #lower = set(data_here.city)
        #ax.fill_between([data_here.day.min(), data_here.day.max()], [1,1]
        matches_here = match_data[match_data.city==city]
        for day, match in zip(matches_here.day, matches_here.match):
            if day >= data_here4.day.min() and day <= data.day.max():
                ax.text(day, ylimup, match, rotation=90, ha='center', va='top', fontsize=10)
        day_start = vac_data[vac_data.city==city].day_start.iloc[0]
        day_end = min(vac_data[vac_data.city==city].day_end.iloc[0], data.day.max())
        if day_end > day_start:
            ax.fill_between([day_start, day_end], [0]*2, [ylimup]*2, color='gray', alpha=.25)
        ax.set_ylim([1e-1, ylimup])

        for holi in holis:
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)
        if city in ['Frankfurt am Main','München','Köln','Düsseldorf','Dortmund','Gelsenkirchen','Stuttgart']:
            holi = date(2024,5,30)
            ax.fill_between([holi-timedelta(1), holi+timedelta(1)], [0]*2, [ylimup]*2, color='C3', alpha=.25)

lg = g._legend
lg.set_title('stadiums')
for tx in lg.texts:
    if tx.get_text() == 'True':
        tx.set_text('with')
    elif tx.get_text() == 'False':
        tx.set_text('without')

plt.savefig(f'plots/fig1_{cdef_alt}/05_contacts_baseline_cmp_horvitz.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig1_{cdef_alt}/05_contacts_baseline_cmp_horvitz.pdf', bbox_inches='tight')
plt.show()